In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import T

chat = ChatOpenAI(temperature=0.1)

In [13]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

In [14]:
template = ChatPromptTemplate.from_messages([
    ("system",
     "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else."),
    ("human", "{question}"),
])

In [15]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

['Pikachu', 'Charmander', 'Bulbasaur', 'Squirtle', 'Jigglypuff']